In [61]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import requests
from StringIO import StringIO
import urllib

import unittest

import sys
import os

import time

caffe_root = '/home/jonnatas/workspace_python/caffe/'
sys.path.insert(0, caffe_root + 'python')

print caffe_root
import caffe


/home/jonnatas/workspace_python/caffe/


In [62]:
caffe.set_mode_cpu()

model_def = os.path.join(caffe_root, 'models', 'bvlc_reference_caffenet','deploy.prototxt')
model_weights = os.path.join(caffe_root, 'models','bvlc_reference_caffenet','bvlc_reference_caffenet.caffemodel')

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)

mu = np.load(os.path.join(caffe_root, 'python','caffe','imagenet','ilsvrc_2012_mean.npy'))
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227



mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [63]:
def printFaces(faces,coordinates):
    print('Total of faces detected = '+str(len(faces)))
    print 'coordinates: '
    
    for coordinate in coordinates:
        x,y,w,h = coordinate
        print [(x,y), (x+w,y+h), (x+w, y), (x, y+h)]
        print '\n'
    print '\n'

In [64]:
def contFaces(faces,frame,DOWNSCALE):
    coordinates = []
    # print 'face detected!'
    for i in faces:
        x, y, w, h = [ v*DOWNSCALE for v in i ]

        coordinates.append((x,y,w,h))
        print x,y,w,h
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
    
    return coordinates
    

In [65]:
def detect(frame):
    height, width, depth = frame.shape
   
    # create grayscale version
    grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # equalize histogram
    cv2.equalizeHist(grayscale, grayscale)
    
    # detect objects
    classifier = cv2.CascadeClassifier("/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml")
    
    DOWNSCALE = 4
    minisize = (frame.shape[1]/DOWNSCALE,frame.shape[0]/DOWNSCALE)
    miniframe = cv2.resize(frame, minisize)
    faces = classifier.detectMultiScale(miniframe)
    coordinates = []
    
    if len(faces)>0:
        coordinates = contFaces(faces,frame,DOWNSCALE)
        #print 'face detected!'
        
    printFaces(faces, coordinates)
    
    return frame, len(faces)

def get_image_url(url):
        cap = cv2.VideoCapture(url)
        ret,img = cap.read()

        return img


def get_image_camera():

        cap = cv2.VideoCapture(0)
        if (cap==True):
            print("deu certo")
            
        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            img = frame.copy()
            # Call the function
            frame = detect(frame)

            # Display the resulting frame
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.imwrite("frame.jpg", frame)
        cv2.imwrite(os.path.join(caffe_root, 'examples', 'images','frame.jpg'), frame)

        
        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()
        cap = cv2

        return frame

In [66]:
def cats_and_dogs_probability(output_prob, labels, path_animal):
        labels_animal = np.loadtxt(path_animal, str, delimiter='\t')
        
        index = 0
        list_animal = []
        probability_animal = 0.0
        
        for i in labels:
                if i in labels_animal:
                        list_animal.append((output_prob[index],labels[index]))
                        probability_animal += output_prob[index]
                index += 1
        list_animal = sorted(list_animal, reverse=True)
        
        for i in list_animal:
            probability , animal_class = i
            print 'Synset ' + str(animal_class)
            print 'Probability ' + str(probability * 100.0) + "%"
        
        return probability_animal

In [81]:
def probability(image):    
        transformed_image = transformer.preprocess('data', image)
        plt.imshow(image)
        plt.axis('off')

        # copy the image data into the memory allocated for the net
        net.blobs['data'].data[...] = transformed_image

        ### perform classification
        output = net.forward()

        output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

        # print 'predicted class is:', output_prob.argmax()
        
        # load ImageNet labels
        labels_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
   
        labels = np.loadtxt(labels_file, str, delimiter='\t')

        dogs_file = os.path.join(caffe_root, 'data','ilsvrc12','dogs.txt')
        cats_file = os.path.join(caffe_root, 'data', 'ilsvrc12','cats.txt')

        top_inds = output_prob.argsort()[::-1][:]

        probability_dogs = cats_and_dogs_probability(output_prob, labels, dogs_file)
        probability_cats = cats_and_dogs_probability(output_prob, labels, cats_file)
               
        print '\n\n::Results::\n'
        print 'Feline probability: ' + str( (probability_cats) * 100.0 ) + "%"
        print 'Canine probability: ' + str( (probability_dogs) * 100.0 ) + "%"
        print 'Most probable synset:', labels[output_prob.argmax()]
        print 'Compatibility: ' + str(output_prob[output_prob.argmax()] * 100.0) + "%"
        print '\n'
        
        return probability_dogs * 100.0 , probability_cats * 100.0 


def options():
        print('1 - image')
        print('2 - url')
        print('3 - camera')
        print('4 - exit')

        return int(raw_input(''))

In [90]:
def option_image(image_name):
        image = cv2.VideoCapture(image_name)
        ret, img = image.read()

        # detect(img)

        image_recognition = caffe.io.load_image(image_name)

        pb = probability(image_recognition)
        image_detected, faces = detect(img)

        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()
        return faces, pb

def option_url(image_url):
        image = get_image_url(image_url)
        
        urllib.urlretrieve(image_url, os.path.join(caffe_root, 'examples', 'images','url.jpg'))
        image_recognition = caffe.io.load_image(os.path.join(caffe_root, 'examples', 'images','url.jpg'))
        # print image
        probability(image_recognition)
        image_detected = detect(image)
        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()
        #return image_de

def option_camera():
        frame = get_image_camera()
        image_recognition = caffe.io.load_image(os.path.join(caffe_root, 'examples', 'images','frame.jpg'))
        probability(image_recognition)
        image_detected = detect(frame)
        
        plt.imshow(image_detected[:,:,::-1])
        plt.title('Detection result')
        plt.axis('off')
        plt.show()

if __name__ == '__main__':
    while True:
        option = options()
        t_init = time.time()
        if option == 1:
            image_name = raw_input('File name:\n')
            option_image(image_name)
        elif option == 2:
            image_url = raw_input('enter the image url:\n')
            option_url(image_url)
        elif option == 3:
            option_camera()
        elif option == 4:
            break
        t_end = time.time()
        print ("Time: "+str(t_end-t_init)+" (s)")
 
    print ("Time of oprion Image: "+str(t_end-t_init)+" (s)")

# Unit Tests

## Teste de desempenho, verificando a velocidade de execução dos metodos.

In [101]:
class TestVelocity(unittest.TestCase):
    def test_file_path(self):
        t_init = time.time()
        option_image('image.jpg')   
        t_end = time.time() 
        t_image = t_end - t_init
        print "Time ="+str(t_image)
        self.assertLessEqual(t_image, 20.00)
        
    def test_option_image(self):
        total_faces, pb = option_image('two_faces.jpg')
        self.assertEqual(total_faces, 2)
    
    def test_option_image_dogs(self):
        total_faces, pb = option_image('cachorro.jpg')
        self.assertGreaterEqual(pb[0], 0.80)
        
    def test_option_image_cats(self):
        total_faces, pb = option_image('cat-300572_960_720.jpg')
        self.assertGreaterEqual(pb[1], 0.80)


In [102]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestVelocity)
unittest.TextTestRunner(verbosity=2,stream=sys.stderr).run(suite)

test_file_path (__main__.TestVelocity) ... 

Synset n02108915 French bulldog
Probability 42.7341908216%
Synset n02085620 Chihuahua
Probability 11.6929322481%
Synset n02096585 Boston bull, Boston terrier
Probability 7.5479760766%
Synset n02091032 Italian greyhound
Probability 6.4249612391%
Synset n02093428 American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier
Probability 5.22337146103%
Synset n02091134 whippet
Probability 2.56597399712%
Synset n02108089 boxer
Probability 1.76141820848%
Synset n02093256 Staffordshire bullterrier, Staffordshire bull terrier
Probability 0.816842913628%
Synset n02110958 pug, pug-dog
Probability 0.588973006234%
Synset n02091244 Ibizan hound, Ibizan Podenco
Probability 0.412018084899%
Synset n02088364 beagle
Probability 0.315974815749%
Synset n02113186 Cardigan, Cardigan Welsh corgi
Probability 0.25468103122%
Synset n02092339 Weimaraner
Probability 0.2218574984%
Synset n02110185 Siberian husky
Probability 0.184753304347%
Synset n02110806 basenji
Probability 0

ok
test_option_image (__main__.TestVelocity) ... 

Time =8.87598609924
Synset n02088364 beagle
Probability 28.1335562468%
Synset n02091032 Italian greyhound
Probability 19.0363511443%
Synset n02089867 Walker hound, Walker foxhound
Probability 5.61018623412%
Synset n02091134 whippet
Probability 3.87849472463%
Synset n02113023 Pembroke, Pembroke Welsh corgi
Probability 3.68739925325%
Synset n02110806 basenji
Probability 2.48413104564%
Synset n02113186 Cardigan, Cardigan Welsh corgi
Probability 1.24913994223%
Synset n02087046 toy terrier
Probability 1.23909758404%
Synset n02088238 basset, basset hound
Probability 1.19849806651%
Synset n02085620 Chihuahua
Probability 1.04333329946%
Synset n02089973 English foxhound
Probability 0.85588209331%
Synset n02110185 Siberian husky
Probability 0.824087299407%
Synset n02107908 Appenzeller
Probability 0.759615376592%
Synset n02095314 wire-haired fox terrier
Probability 0.756680639461%
Synset n02093256 Staffordshire bullterrier, Staffordshire bull terrier
Probability 0.652977544814%
Synset n02093428 A

ok
test_option_image_cats (__main__.TestVelocity) ... 

Synset n02114855 coyote, prairie wolf, brush wolf, Canis latrans
Probability 0.000100405407011%
Synset n02114367 timber wolf, grey wolf, gray wolf, Canis lupus
Probability 4.21002184225e-06%
Synset n02115913 dhole, Cuon alpinus
Probability 2.41570852211e-06%
Synset n02105855 Shetland sheepdog, Shetland sheep dog, Shetland
Probability 2.88064905263e-07%
Synset n02109961 Eskimo dog, husky
Probability 1.56362067738e-07%
Synset n02115641 dingo, warrigal, warragal, Canis dingo
Probability 1.38879130418e-07%
Synset n02114548 white wolf, Arctic wolf, Canis lupus tundrarum
Probability 6.15576922769e-08%
Synset n02110185 Siberian husky
Probability 4.15806583609e-08%
Synset n02094258 Norwich terrier
Probability 3.86077336767e-08%
Synset n02110806 basenji
Probability 3.74715342089e-08%
Synset n02114712 red wolf, maned wolf, Canis rufus, Canis niger
Probability 3.29282212608e-08%
Synset n02113023 Pembroke, Pembroke Welsh corgi
Probability 1.95776533873e-08%
Synset n02112018 Pomeranian
Probability 

ok
test_option_image_dogs (__main__.TestVelocity) ... 

Synset n02106550 Rottweiler
Probability 98.9165604115%
Synset n02089078 black-and-tan coonhound
Probability 0.609203241765%
Synset n02107142 Doberman, Doberman pinscher
Probability 0.115914631169%
Synset n02101006 Gordon setter
Probability 0.103879941162%
Synset n02108551 Tibetan mastiff
Probability 0.0296359794447%
Synset n02107312 miniature pinscher
Probability 0.0293965189485%
Synset n02088632 bluetick
Probability 0.0280844018562%
Synset n02107908 Appenzeller
Probability 0.018620704941%
Synset n02099712 Labrador retriever
Probability 0.0166132784216%
Synset n02088466 bloodhound, sleuthhound
Probability 0.0131461580168%
Synset n02112706 Brabancon griffon
Probability 0.0116526614875%
Synset n02105412 kelpie
Probability 0.0104577673483%
Synset n02108000 EntleBucher
Probability 0.00967595915427%
Synset n02093428 American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier
Probability 0.00671904490446%
Synset n02108422 bull mastiff
Probability 0.0052

ok

----------------------------------------------------------------------
Ran 4 tests in 33.279s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Testes das unitarios